Converting between NetworkX and Apache AGE (GraphDB) involves transforming the graph structure and properties to match the data model of the respective graph database. Here's some information on converting NetworkX to Apache AGE GraphDB and vice versa:

## Connect with AGE

In [1]:
import psycopg2
from age.Networkx import *

# set DB path and graph name
conn = psycopg2.connect(
    host="localhost",
    port="5430",
    dbname="postgresDB",
    user="postgresUser",
    password="postgresPW")
graphName = 'bitnine_global_inic'

## Networkx to AGE

### Create a Networkx Directed Graph with some nodes and edges

In [2]:
# Create an empty graph
G = nx.DiGraph()

# Add some nodes
G.add_node('user1',label='User',properties={'name':'Moontasir'})
G.add_node('user2',label='User',properties={'name':'Sanjida'})

G.add_node('movie1',label='Movie',properties={'name':'The Shawshank Redemption'})
G.add_node('movie2',label='Movie',properties={'name':'The Godfather'})
G.add_node('movie3',label='Movie',properties={'name':'The Dark Knight'})
G.add_node('movie4',label='Movie',properties={'name':'Army of Thieves'})
G.add_node('movie5',label='Movie',properties={'name':'Oceans Thirteen'})
G.add_node('movie6',label='Movie',properties={'name':'Heat'})

G.add_node('genre1',label='Genre',properties={'name':'Drama'})
G.add_node('genre2',label='Genre',properties={'name':'Crime'})
G.add_node('genre3',label='Genre',properties={'name':'Action'})

# Add some edges
G.add_edge('user1','movie1', label='Watched')
G.add_edge('user1','movie2', label='Watched')
G.add_edge('user2','movie4', label='Watched')
G.add_edge('movie1','genre1',label='Belongs_to')
G.add_edge('movie4','genre1',label='Belongs_to')
G.add_edge('movie2','genre2',label='Belongs_to')
G.add_edge('movie5','genre2',label='Belongs_to')
G.add_edge('movie3','genre3',label='Belongs_to')
G.add_edge('movie6','genre3',label='Belongs_to')


### Networkx Graph to AGE
- Add all Nodes and Edges
- Every Nodes and Edges must have `label`

In [3]:
# Making sure working with new Graph
age.setUpAge(conn, graphName)
age.deleteGraph(conn, graphName)

networkxToAge(conn, G, graphName=graphName, isPrint=False)

## AGE to Networkx

### Get All Nodes and Edges in Networkx Directed Graph

In [4]:
G = ageToNetworkx(conn,graphName)

Found 11 nodes and 9 edges


### Get Nodes and Edges From custom query
Create a networkx graph with the scenario as follow
- Add all user
- add watched movies
- Add relation between user and watched movies

In [5]:
# Add All User
G = ageToNetworkx(conn,
                  graphName,
                  query="""
                    SELECT * from cypher('%s', $$
                        MATCH (n:User)   
                        RETURN n
                    $$) as (n agtype);
                    """ % (graphName), 
                  isPrint=False)

# Add Watched Movies
G = ageToNetworkx(conn,
                  graphName,
                  G=G,
                  query="""
                    SELECT * from cypher('%s', $$
                        MATCH ()-[n:Watched]->()
                        RETURN n
                    $$) as (n agtype);
                    """ % (graphName),
                  isPrint=False)

print(G)

DiGraph with 5 nodes and 3 edges
